In [99]:
import time
import pprint
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
    

In [71]:
def create_df(soup):
    df = pd.DataFrame(columns=['Name','Location','URL'])
    cards = soup.findAll('div', attrs = {'class':'styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP'})
    for trail in cards:
        url = 'https://www.alltrails.com/'+trail.attrs['itemid']
        name = trail.find('a', attrs = {'class':'xlate-none styles-module__link___12BPT'})['title']
        loc = trail.find('a', attrs = {'class':'xlate-none styles-module__location___11FHK styles-module__info___1Mbn6 styles-module__link___3T9FO'})['title']
        row = pd.Series([name,loc,url])
        df = df.append(row, ignore_index=True)
    return df
        
    

In [117]:
def create_db(soup,driver):
    urls=[]
    cards = soup.findAll('div', attrs = {'class':'styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP'})
    for trail in cards:
        urls.append('https://www.alltrails.com/'+trail.attrs['itemid'])

    df = get_trail_info(urls,driver)
    return df

In [144]:
def page_parser(page,url):
    
    
    #header
    header = page.find('div',id='title-and-menu-box')
    trail_name = header.find('h1').text
    difficulty = header.find('span').text
    rating = header.find('meta', attrs = {'itemprop':'ratingValue'})
    reviews = header.find('meta', attrs = {'itemprop':'reviewCount'}).text
    
#     loc = trail.find('a', attrs = {'class':'xlate-none styles-module__location___11FHK styles-module__location___3wEnO'})['title']
    loc = header.find('a', attrs = {'class':'xlate-none styles-module__location___11FHK styles-module__location___3wEnO'})['title']
    
    #trail stats section
    trail_stats = page.findAll('span', attrs={'class':'styles-module__detailData___kQ-eK'})
    length = trail_stats[0].text
    elev_gain = trail_stats[1].text
    route = trail_stats[2].text
    
    #metadata
#     lat = page.find('meta', attrs = {'itemprop':'latitude'})
#     long = page.find('meta', attrs = {'itemprop':'longitude'})
    lat = page.findAll('meta', attrs = {'itemprop':'latitude'})
    long = page.findAll('meta', attrs = {'itemprop':'longitude'})
    
    row = pd.Series([trail_name,loc,rating,reviews,difficulty,length,elev_gain,route,lat,long,url], 
                    index=['Name','Location','Rating','Reviews','Difficulty','Length','Elevation gain','Route type','lat','long','URL'])
    return row

In [138]:
def get_trail_info(urls,driver):
    df = pd.DataFrame(columns=['Name','Location','Rating','Reviews','Difficulty','Length','Elevation gain','Route type','lat','long','URL'])
    wait = WebDriverWait(driver,20)
#     for url in urls:
#         driver.get(url)
#         time.sleep(10)
#         row = page_parser(BeautifulSoup(driver.page_source,'html.parser'),url)
#         df = df.append(row, ignore_index=True)
    driver.get(urls[0])
    time.sleep(10)
    row = page_parser(BeautifulSoup(driver.page_source,'html.parser'),urls[0])
    df=df.append(row, ignore_index=True)
    return df
            
        

In [5]:
def get_trails(driver):
    url = 'https://www.alltrails.com/us/colorado'
    driver.get(url)
    wait = WebDriverWait(driver,20)

    while True:
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.styles-module__button___1nuva')))
            show_more.click()
            time.sleep(1)
        except:
            break
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

In [6]:
un = 'timrchilders@gmail.com'
pwd = 'galvanize'

# options = Options()
# options.add_argument("--headless")
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.privatebrowsing.autostart", True)


driver=webdriver.Firefox(executable_path="/usr/bin/geckodriver", firefox_profile=firefox_profile)

soup = get_trails(driver)
driver.quit()

In [27]:
print(soup.title)

<title class="xlate-none">  Best trails in Colorado
 | AllTrails</title>


In [69]:
trails = create_df(soup)
print(f"Scraped {len(trails)} trails")

Scraped 1000 trails


In [86]:
trails

,0,1,2
0,Emerald Lake Trail,Rocky Mountain National Park,https://www.alltrails.com//trail/us/colorado/e...
1,Royal Arch Trail,Boulder Open Space,https://www.alltrails.com//trail/us/colorado/r...
2,Sky Pond via Glacier Gorge Trail,Rocky Mountain National Park,https://www.alltrails.com//trail/us/colorado/s...
3,Chavez and Beaver Brook Trail Loop,Genesee Park,https://www.alltrails.com//trail/us/colorado/b...
4,The Loch via Glacier Gorge Trail,Rocky Mountain National Park,https://www.alltrails.com//trail/us/colorado/t...
...,...,...,...
995,Bear Creek Trail,American Flats Wilderness Area,https://www.alltrails.com//trail/us/colorado/b...
996,Boreas Pass Road and Baker's Tank Trail Loop,Ridgeview Park,https://www.alltrails.com//trail/us/colorado/b...
997,Balanced Rock Trail,Pike National Forest,https://www.alltrails.com//trail/us/colorado/b...
998,"Village Walk, Red Rocks and Cherry Gulch Trail...",Matthews Winters Park,https://www.alltrails.com//trail/us/colorado/v...


In [145]:
# firefox_options = Options()
# firefox_options.add_argument("--headless")
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.privatebrowsing.autostart", True)


driver=webdriver.Firefox(executable_path="/usr/bin/geckodriver", firefox_profile=firefox_profile )
db = create_db(soup, driver)
driver.close()

In [146]:
db

,Name,Location,Rating,Reviews,Difficulty,Length,Elevation gain,Route type,lat,long,URL
0,Emerald Lake Trail,Rocky Mountain National Park,,,moderate,3.2 mi,698 ft,Out & back,[[]],[[]],https://www.alltrails.com//trail/us/colorado/e...
